# KL2 Data Processing & Analysis

In [1]:
%load_ext autoreload
%autoreload 2

## 1 Relabeling

### 1.1 Features Renaming & Preliminary Relabeling

The folder "Data Preprocessing" defines some rules to rename features as well as some preliminary relabeling of feature values.

This section load these rules and process the data as a first step.

In [2]:
# libraries, utility functions

import sys
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob

def load_lookup_table(folder_path):
    '''
    "Data Preprocessing" folder contains a bunch of .xlsx's, each coming with one or multiple sheets.
    Each sheet is expected to have only 2 column.
    The first row is a mapping from original column name in collected data, to a more refined (?) variation.
    The rest of the rows are a mapping from field values on the questionnaire to an interpretable label, usually a string.
    If for some row there's no value at the second column, that means we are not interested in those feature values.
    '''
    column_map = {}
    lookup_table = {'columns': column_map} # `column_map`: map old column in original

    excel_paths = glob(os.path.join(folder_path, "*.xlsx"))
    for path in excel_paths:    # read each excel files
        xls = pd.ExcelFile(path)
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(path, sheet_name=sheet_name, header=None)    # one df per sheet
            if df.shape[1] >= 2:  # Ensure at least two columns exist
                old_col_name, new_col_name = df.iloc[0, :2].str.strip()
                if pd.isna(old_col_name):
                    continue
                if pd.isna(new_col_name):
                    new_col_name = old_col_name
                column_map[old_col_name] = new_col_name
                relabel_map = dict(df.iloc[1:, :2].itertuples(index=False))
                lookup_table[new_col_name] = relabel_map
    # more: add two new feature names: 'diagnosis_b' and 'diagnosis_c'
    lookup_table['columns']['diagnosisb'] = 'diagnosis_b'
    lookup_table['columns']['diagnosisc'] = 'diagnosis_c'
    # set the mapping to the same as 'diagnosis'
    lookup_table['diagnosis_b'] = lookup_table['diagnosis']
    lookup_table['diagnosis_c'] = lookup_table['diagnosis']
        
    return lookup_table

def format_data(raw_data_path, lookup_table, keep_columns=True):
    df = pd.read_csv(raw_data_path, delimiter='|', low_memory=False)[:-1] # drop last row: summarizing # rows of the table
    df.iloc[:, 0] = df.iloc[:, 0].astype(float) # first column is coerced to int due to error caused by last special row 
    df.rename(columns=lookup_table['columns'], inplace=True)
    df.replace(lookup_table, inplace=True)
    if not keep_columns:
        columns = df.columns.intersection(lookup_table['columns'].values())
        df = df[columns]

    return df

# # Usage
# folder_path = "../data/Data Preprocessing/"
# raw_data_path = '../data/Original files/Cogan_eRD_RIC1_request.txt'  # Update this to your raw data file path
# output_path = '/path/to/output/renamed_data.xlsx'  # Update this to your desired output file path

# data_dict = load_data_dictionary(folder_path)
# relabeled_data = rename_columns(raw_data_path, data_dict, keep_columns=True)  # Set keep_columns to False if you want to drop columns not in the data dictionary

In [ ]:
# load folder "Data Preprocessing" as a lookup table

folder_path = "./data/Data Preprocessing/"
raw_data_path = './data/Original files/Cogan_eRD_RIC1_request.txt'
lookup_table = load_lookup_table(folder_path)

lookup_table['columns']

In [ ]:
# preprocess data with lookup table

df = format_data(raw_data_path, lookup_table, keep_columns=True)

df

In [ ]:
# have a look at unique diagnosis
print("# of unique diagnosis:", df['diagnosis'].unique().size)
df['diagnosis'].unique()

In [6]:
# save preprocessed data

df.to_csv('./data/Cleaned files/Cogan_1_1.csv', index=False)

In [7]:
# save lookup table
import pickle

with open("./data/Cleaned files/lookup_table.pkl", 'wb') as f:
    pickle.dump(lookup_table, f)

#### Examine some features of interest here

In [ ]:
lookup_table['language']

### 1.2 Add Aggregated Outcomes 

In [6]:
# load formated dataframe and see all feature names

import pandas as pd
import pickle
from tabulate import tabulate
import numpy as np

# always start from loading the CSV file from previous section
df = pd.read_csv('./data/Cleaned files/Cogan_1_1.csv', low_memory=False)
with open("./data/Cleaned files/lookup_table.pkl", 'rb') as f:
    lookup_table = pickle.load(f)

feature_names = [*df.head()]
print("total # of features:", len(feature_names))

print(tabulate(enumerate(feature_names), headers=["Index", "Name"], tablefmt="grid"))

total # of features: 449
+---------+--------------------------------+
|   Index | Name                           |
+=========+================================+
|       0 | sex                            |
+---------+--------------------------------+
|       1 | marital_status                 |
+---------+--------------------------------+
|       2 | admityear                      |
+---------+--------------------------------+
|       3 | admitclass                     |
+---------+--------------------------------+
|       4 | admit_from                     |
+---------+--------------------------------+
|       5 | prehospital_living             |
+---------+--------------------------------+
|       6 | prehospital_living_with        |
+---------+--------------------------------+
|       7 | payor_primary                  |
+---------+--------------------------------+
|       8 | payor_secondary                |
+---------+--------------------------------+
|       9 | impgroupadmit     

In [7]:
# define outcome features
selfcare_items = [
    "eating",
    "oral",
    "toileting",
    "bathe",
    "dress_upper",
    "dress_lower",
    "footwear",
]
mobility_items = [
    "roll_lr",
    "sit_lying",
    "lying_sit",
    "sit_stand",
    "bed_chair",
    "toilet_trans",
    "car_trans",
    "walk10ft",
    "walk50ft",
    "walk150ft",
    "walk10ft_uneven",
    "walk1step",
    "walk4step",
    "walk12step",
    "pickup"
]
print("# of selfcare items:", len(selfcare_items))
print("# of mobility items:", len(mobility_items))

# of selfcare items: 7
# of mobility items: 15


In [8]:
# define ordinal sum to measure mapping
selfcare_table = {
    7: 19.65,
    8: 24.7,
    9: 27.64,
    10: 29.69,
    11: 31.44,
    12: 33.02,
    13: 34.46,
    14: 35.78,
    15: 36.97,
    16: 38.04,
    17: 38.98,
    18: 39.84,
    19: 40.62,
    20: 41.35,
    21: 42.05,
    22: 42.73,
    23: 43.39,
    24: 44.07,
    25: 44.77,
    26: 45.52,
    27: 46.33,
    28: 47.25,
    29: 48.33,
    30: 49.64,
    31: 51.28,
    32: 53.37,
    33: 56.1,
    34: 59.86,
    35: 65.47,
    36: 71.86,
    37: 76.96,
    38: 81.08,
    39: 84.76,
    40: 88.57,
    41: 93.54,
    42: 100.64,
}
mobility_table = {
    15: 4.86,
    16: 11.07,
    17: 14.3,
    18: 16.1,
    19: 17.35,
    20: 18.34,
    21: 19.16,
    22: 19.89,
    23: 20.54,
    24: 21.14,
    25: 21.71,
    26: 22.26,
    27: 22.79,
    28: 23.3,
    29: 23.81,
    30: 24.31,
    31: 24.81,
    32: 25.31,
    33: 25.82,
    34: 26.32,
    35: 26.82,
    36: 27.32,
    37: 27.83,
    38: 28.33,
    39: 28.82,
    40: 29.32,
    41: 29.81,
    42: 30.29,
    43: 30.77,
    44: 31.25,
    45: 31.72,
    46: 32.18,
    47: 32.65,
    48: 33.11,
    49: 33.57,
    50: 34.02,
    51: 34.48,
    52: 34.93,
    53: 35.39,
    54: 35.85,
    55: 36.32,
    56: 36.88,
    57: 37.28,
    58: 37.77,
    59: 38.28,
    60: 38.81,
    61: 39.36,
    62: 39.94,
    63: 40.56,
    64: 41.21,
    65: 41.92,
    66: 42.68,
    67: 43.52,
    68: 44.46,
    69: 45.53,
    70: 46.75,
    71: 48.17,
    72: 49.83,
    73: 51.76,
    74: 53.98,
    75: 56.42,
    76: 58.93,
    77: 61.36,
    78: 63.64,
    79: 65.73,
    80: 67.83,
    81: 69.82,
    82: 71.78,
    83: 73.76,
    84: 75.79,
    85: 77.91,
    86: 80.21,
    87: 82.62,
    88: 86.36,
    89: 90.90,
    90: 98.38
}


In [9]:
# safe buffer
last_df = df

In [13]:
# for outcome features, replace invalid responses as nan and count invalid samples
df = last_df.copy()
selfcare_columns = [item + '_adm' for item in selfcare_items] + [item + '_dc' for item in selfcare_items]
mobility_columns = [item + '_adm' for item in mobility_items] + [item + '_dc' for item in mobility_items]

subset_columns = [item + '_adm' for item in selfcare_items+mobility_items]
subset_columns += [item + '_dc' for item in selfcare_items+mobility_items]
df.loc[:, subset_columns] = df.loc[:, subset_columns].replace({
    'Refused': np.nan,
    'Not_applicable': np.nan,
    'Not_attempted': np.nan,
    'Safety': np.nan
})
print(df[subset_columns].isna().sum(axis=0))
# df = df.dropna(subset=selfcare_columns)
# df = df.dropna(subset=subset_columns)
# print("# of valid patients changed from", len(last_df), "to", len(df))

eating_adm              2474
oral_adm                1387
toileting_adm            984
bathe_adm               4420
dress_upper_adm         1750
dress_lower_adm         1343
footwear_adm             958
roll_lr_adm             1229
sit_lying_adm           1430
lying_sit_adm           1433
sit_stand_adm           3711
bed_chair_adm           2403
toilet_trans_adm        5618
car_trans_adm          24704
walk10ft_adm           15365
walk50ft_adm           24103
walk150ft_adm          32291
walk10ft_uneven_adm    31558
walk1step_adm          25778
walk4step_adm          29097
walk12step_adm         37282
pickup_adm             29537
eating_dc               3732
oral_dc                 3152
toileting_dc            3077
bathe_dc                3336
dress_upper_dc          3124
dress_lower_dc          3099
footwear_dc             3135
roll_lr_dc              3199
sit_lying_dc            3145
lying_sit_dc            3142
sit_stand_dc            3553
bed_chair_dc            3125
toilet_trans_d

In [14]:
# calculate 4 measures

DO_IMPUTE_MISSING_OUTCOMES = True

if DO_IMPUTE_MISSING_OUTCOMES:
    df.loc[:, subset_columns] = df.loc[:, subset_columns].fillna(1) # replace NaNs with 1 ordinal

inverse_lookup = {v:k for k,v in lookup_table[selfcare_items[0]+'_adm'].items()}
print(inverse_lookup)

selfcare_sum_adm = df[[item + '_adm' for item in selfcare_items]].replace(inverse_lookup).sum(axis=1, skipna=False)
# print(selfcare_sum_adm[:10])
selfcare_sum_dc = df[[item + '_dc' for item in selfcare_items]].replace(inverse_lookup).sum(axis=1, skipna=False)
print("selfcare sum adm nan count:", np.sum(np.isnan(selfcare_sum_adm)))
print("selfcare sum dc nan count:", np.sum(np.isnan(selfcare_sum_dc)))

selfcare_measure_adm = selfcare_sum_adm.map(selfcare_table)
# print(selfcare_measure_adm[:10])
selfcare_measure_dc = selfcare_sum_dc.map(selfcare_table)
print("selfcare measure adm nan count:", np.sum(pd.isna(selfcare_measure_adm).to_numpy()))
print("selfcare measure dc nan count:", np.sum(pd.isna(selfcare_measure_dc).to_numpy()))

mobility_sum_adm = df[[item + '_adm' for item in mobility_items]].replace(inverse_lookup).sum(axis=1, skipna=False)
mobility_sum_dc = df[[item + '_dc' for item in mobility_items]].replace(inverse_lookup).sum(axis=1, skipna=False)
print("mobility sum adm nan count:", np.sum(np.isnan(mobility_sum_adm)))
print("mobility sum dc nan count:", np.sum(np.isnan(mobility_sum_dc)))
mobility_measure_adm = mobility_sum_adm.map(mobility_table)
mobility_measure_dc = mobility_sum_dc.map(mobility_table)
print("mobility measure adm nan count:", np.sum(pd.isna(mobility_measure_adm).to_numpy()))
print("mobility measure dc nan count:", np.sum(pd.isna(mobility_measure_dc).to_numpy()))

{'Independent': 6, 'Setup': 5, 'Supervised': 4, 'Mod_assist': 3, 'Max_assist': 2, 'Dependent': 1, 'Refused': 7, 'Not_applicable': 9, 'Not_attempted': 10, 'Safety': 88}
selfcare sum adm nan count: 8035
selfcare sum dc nan count: 4230
selfcare measure adm nan count: 8035
selfcare measure dc nan count: 4230
mobility sum adm nan count: 39792
mobility sum dc nan count: 24421
mobility measure adm nan count: 39792
mobility measure dc nan count: 24421


In [ ]:
# add outcome measure columns to the df
df['selfcare_measure_adm'] = selfcare_measure_adm
df['selfcare_measure_dc'] = selfcare_measure_dc
df['mobility_measure_adm'] = mobility_measure_adm
df['mobility_measure_dc'] = mobility_measure_dc

# add two delta measure columns, just for quick filtering out valid samples (measures @adm&dc are both valid)
df['selfcare_measure_delta'] = (selfcare_measure_dc - selfcare_measure_adm).round(2)
df['mobility_measure_delta'] = (mobility_measure_dc - mobility_measure_adm).round(2)
print("# of invalid samples for selfcare:", np.isnan(df['selfcare_measure_delta']).sum(), " out of", len(df))
print("# of invalid samples for mobility:", np.isnan(df['mobility_measure_delta']).sum(), " out of", len(df))

In [16]:
# save processed data with 6 extra columns

df.to_csv('./data/Cleaned files/Cogan_1_2.csv', index=False)

### 1.3 Relabel & Precess Features


In [52]:
# total # of features and a list of all feature names

import pandas as pd
import pickle
import numpy as np
from tabulate import tabulate

# always start from loading the CSV file from previous section
df = pd.read_csv('./data/Cleaned files/Cogan_1_2.csv', low_memory=False)
with open("./data/Cleaned files/lookup_table.pkl", 'rb') as f:
    lookup_table = pickle.load(f)

feature_names = [*df.head()]
feature_dtypes = df.dtypes
print("total # of features:", len(feature_names))

print(tabulate([(index, name, feature_dtypes[name]) for index, name in enumerate(feature_names)], headers=["Index", "Name", "Dtype"], tablefmt="grid"))
categorical_feature_set = set()
numeric_feature_set = set()
ranked_int_feature_set = set()
binary_feature_set = set()

total # of features: 455
+---------+--------------------------------+---------+
|   Index | Name                           | Dtype   |
+=========+================================+=========+
|       0 | sex                            | object  |
+---------+--------------------------------+---------+
|       1 | marital_status                 | object  |
+---------+--------------------------------+---------+
|       2 | admityear                      | float64 |
+---------+--------------------------------+---------+
|       3 | admitclass                     | float64 |
+---------+--------------------------------+---------+
|       4 | admit_from                     | object  |
+---------+--------------------------------+---------+
|       5 | prehospital_living             | object  |
+---------+--------------------------------+---------+
|       6 | prehospital_living_with        | object  |
+---------+--------------------------------+---------+
|       7 | payor_primary               

In [53]:
# set column 'id' as index
df['id'] = df['id'].astype(int)

df.set_index('id', inplace=True)
print(df)

          sex marital_status  admityear  admitclass  \
id                                                    
1      female        Married     2023.0         1.0   
2        male        Married     2023.0         1.0   
3        male        Married     2023.0         1.0   
4      female        Widowed     2022.0         1.0   
5      female            NaN     2023.0         1.0   
...       ...            ...        ...         ...   
43741  female        Widowed     2022.0         1.0   
43742  female  Never married     2022.0         NaN   
43743  female        Married     2022.0         1.0   
43744    male        Married     2022.0         1.0   
43745  female        Widowed     2023.0         1.0   

                        admit_from prehospital_living prehospital_living_with  \
id                                                                              
1      Short-term general hospital               Home                  Family   
2      Short-term general hospital       

#### Column: "language"

In [54]:
# safe buffer
last_df = df

In [55]:
# process column "language": English or Non-English
df = last_df.copy() # always work on a fresh copy of last step's output

categorical_feature_set.add('language')

print(df['language'].unique())
df['language'] = df['language'].str.contains('eng|enlish', case=False, na=False)
df['language'] = df['language'].map({True: 'English', False: 'Non-English'})
print(df['language'].unique())

['English' 'ENGLISH' 'Spanish' 'Other' 'eng' 'english' 'spanish' 'SPANISH'
 'Vietnamese' 'Unknown' 'Polish' 'English and Spanish' 'Creole' 'Eng'
 'MANDARIN' 'Cambodia' 'Chinese' 'Korean' 'Englissh' 'HINDI' 'Enlglish'
 'POLISH' 'Sindhi/Urdu then English' 'French' 'Farsi' 'Panjabi' 'Mandarin'
 'Japanese' 'Malayalam' 'Ghana' 'Cambodian' 'ENLGISH' 'Indonesian'
 'tagalog' 'Swahili' 'Estonian' 'Portuguese' 'Cantonese'
 'Chinese, Mandarin' 'gujarati' 'German' 'other' 'Hmong' 'English/Spanish'
 'Russian' 'Englsih' 'Karen' 'Arabic' 'CREOLE' 'Undetermined' 'Gujarati'
 'Urdu' 'BANGLADESH' 'Hindi' 'Tagalog' 'TOISAN' 'Spanish and English'
 'Chinese - Canto' 'korean' 'creole' 'ASL' 'WHITE' 'Italian'
 'ENGLISH/TURKISH' 'Haitian' 'TAGALOG' 'en' 'CANTONESE' 'ENglish'
 'CHINESE' 'Serbian' 'Ukrainian' 'Persian' 'Swedish' 'English, Spanish'
 'Enbglish' 'Bosnian' 'Albanian' 'LAO' 'chinese' 'Chinese - Manda'
 'Slovak' 'French Creole' 'English, Other' 'ukranian' 'ENGLSIH' 'Enlish'
 'Chuukese' 'VIETNAMESE' 'A

#### Column: "hispanic"

In [56]:
# safe buffer
last_df = df

In [57]:
# process column "hispanic" (ethnicity): English or Non-English
df = last_df.copy() # always work on a fresh copy of last step's output

categorical_feature_set.add('hispanic')
print("nan count before:", pd.isna(df['hispanic']).sum())

# 1. fill 'Yes' where any of the hispanic subcategories are 'Yes'
hispanic_yes_cols = ['hispanic_mexican', 'hispanic_puertorican', 'hispanic_cuban', 'hispanic_other']
df.loc[df['hispanic'].isna() & (df[hispanic_yes_cols] == 'Yes').any(axis=1), 'hispanic'] = 'Yes'

# 2. fill 'No' where hispanic_unable or hispanic_declined are 'Yes'
hispanic_no_cols = ['hispanic_unable', 'hispanic_declined']
df.loc[df['hispanic'].isna() & (df[hispanic_no_cols] == 'Yes').any(axis=1), 'hispanic'] = 'No'

# 3. for rows where 'hispanic' is still NaN, fill it with 'No'
df['hispanic'].fillna('No', inplace=True)

print("nan count after:", pd.isna(df['hispanic']).sum())
print(df['hispanic'].unique())


nan count before: 4514
nan count after: 0
['No' 'Yes']


#### Column: "race"

In [58]:
# safe buffer
last_df = df

In [59]:
# process column "race"
df = last_df.copy() # always work on a fresh copy of last step's output

categorical_feature_set.add('race')

# grouping east asians (chinese, japanese, korean)
race_is_east_asian = df[['race_chinese', 'race_japanese', 'race_korean']].eq('Yes').any(axis=1) # bool column

# grouping other asians (filipino, vietnamese, asian other)
race_is_asian_other = df[['race_filipino', 'race_vietnamese', 'race_asianother']].eq('Yes').any(axis=1)

# grouping pacific islanders (hawaiian, guam, samoan, pacific islander other)
race_is_pacific_islander = df[['race_hawaiian', 'race_guam', 'race_samoan', 'race_pacificislander_other']].eq('Yes').any(axis=1)

# asian indian mapping
race_is_asian_indian = df['race_asianindian'].eq('Yes')

# grouping unknown (unable, declined, none of the above)
race_is_unknown = df[['race_unable', 'race_declined', 'race_noneoftheabove']].eq('Yes').any(axis=1)

# keeping the original race columns (race_white, race_black, race_alaskan) as is but casting them to binary
race_is_white = df['race_white'].eq('Yes')
race_is_black = df['race_black'].eq('Yes')
race_is_alaskan = df['race_alaskan'].eq('Yes')

df['race'] = np.nan
df['race'] = df['race'].astype('object')

df.loc[race_is_white, 'race'] = 'White'
df.loc[race_is_black, 'race'] = 'Black'
df.loc[race_is_alaskan, 'race'] = 'Alaskan'
df.loc[race_is_asian_indian, 'race'] = 'Asian_Indian'
df.loc[race_is_east_asian, 'race'] = 'East_Asian'
df.loc[race_is_asian_other, 'race'] = 'Southeast_Asian'
df.loc[race_is_pacific_islander, 'race'] = 'Pacific_islander'
df.loc[race_is_unknown, 'race'] = 'Unknown'

# fill any remaining NaNs with 'Unknown' if no condition matched
df['race'].fillna('Unknown', inplace=True)

print("nan count after:", pd.isna(df['race']).sum())
print(df['race'].unique())


nan count after: 0
['White' 'Black' 'Unknown' 'Southeast_Asian' 'Alaskan' 'Asian_Indian'
 'East_Asian' 'Pacific_islander']


#### Column: all bool features
**NaNs**: There shouldn't be any NaN left after processing.

In [60]:
# safe buffer
last_df = df

In [61]:
# bool features are generally recorded as "Yes" / "No" in raw data; covert to 0 / 1
df = last_df.copy() # always work on a fresh copy of last step's output

binary_features = [
    "transport_lack_med",
    "transport_lack_nonmed",
    "conduct_bims",
    "conduct_sams",
    "sams_season",
    "sams_room",
    "sams_names",
    "sams_hosp",
    "sams_none_above",
    "acute_mental_change",
    "wc_manual_prior",
    "wc_motor_prior",
    "mechlift_prior",
    "walker_prior",
    "orth_pros_prior",
    "no_device_prior",
    "wc_user",
    "pvd_comorbid",
    "diabetes_comorbid",
    "nutrition_parenteral_adm",
    "nutrition_tube_adm",
    "nutrition_mech_diet_adm",
    "nutrition_ther_diet_adm",
    "antipsychotic_taking_adm",
    "antipsychotic_ind_adm",
    "anticoagulant_taking_adm",
    "anticoagulant_ind_adm",
    "antibiotic_taking_adm",
    "antibiotic_ind_adm",
    "opioid_taking_adm",
    "opioid_ind_adm",
    "antiplatelet_taking_adm",
    "antiplatelet_ind_adm",
    "hypoglycemic_taking_adm",
    "hypoglycemic_ind_adm",
    "chemo_adm",
    "chemo_iv_adm",
    "chemo_oral_adm",
    "chemo_other_adm",
    "radiation_adm",
    "oxygen_adm",
    "oxygen_cont_adm",
    "oxygen_int_adm",
    "oxygen_high_adm",
    "suctioning_adm",
    "suctioning_sched_adm",
    "suctioning_asneeded_adm",
    "trach_adm",
    "vent_invasive_adm",
    "vent_noninvasive_adm",
    "vent_bipap_adm",
    "vent_cpap_adm",
    "meds_iv_adm",
    "meds_iv_vasoactive_adm",
    "meds_iv_antibiotic_adm",
    "meds_iv_anticoagulant_adm",
    "meds_iv_other_adm",
    "transfusions_adm",
    "dialysis_adm",
    "hemodialysis_adm",
    "peritoneal_dialysis_adm",
    "iv_access_adm",
    "iv_access_periph_adm",
    "iv_access_mid_adm",
    "iv_access_cent_adm",
]
for name in binary_features:
    binary_feature_set.add(name)
    print(f"----------working on: {name}----------")
    df[name] = df[name].apply(lambda x: x.lower() if isinstance(x, str) else x) # lower-casify first
    uniques = set(df[name].unique())
    print("uniques before:", uniques)
    print("dtype before:", df[name].dtype)
    print("nan count before:", pd.isna(df[name]).sum())
    if uniques == {0, 1}: # already processed
        continue
    if name == "conduct_sams":
        df[name].fillna("yes", inplace=True)
        df[name].replace({
            "yes": 1,
            "no": 0,
        }, inplace=True)
    elif name in {"sams_season","sams_room","sams_names","sams_hosp"}: # default: yes for all those completed BIMS
        df[name].replace({
            "no": 0,
            "yes": 1,
            np.nan: 1,
        }, inplace=True)
        df[name] = df[name].astype('int64')
    elif name == "sams_none_above":
        df[name].replace({
            "no": 0,
            "yes": 1,
            np.nan: 0,
        }, inplace=True)
        df[name] = df[name].astype('int64')
    elif '_ind_' in name:
        df[name].replace({
            "no": 0,
            "yes": 1,
            np.nan: 0,
        }, inplace=True)
        df[name] = df[name].astype('int64')
    else: # default behavior
        if uniques == {np.nan, "yes"}:
            df[name].fillna("no", inplace=True)
        elif uniques == {np.nan, "no"}:
            df[name].fillna("yes", inplace=True)
        elif uniques != {"yes", "no"}:
            raise Exception(f"{name} is not a binary feature")
        df[name].replace({
            "yes": 1,
            "no": 0,
        }, inplace=True)
    print("uniques after:", df[name].unique())
    if set(df[name].unique()) != {0, 1}:
        raise Exception(f"{name} is not well-formated, double check!")
    print("dtype after:", df[name].dtype)
    print("nan count after:", pd.isna(df[name]).sum())

----------working on: transport_lack_med----------
uniques before: {'yes', nan}
dtype before: object
nan count before: 42493
uniques after: [0 1]
dtype after: int64
nan count after: 0
----------working on: transport_lack_nonmed----------
uniques before: {'yes', nan}
dtype before: object
nan count before: 42725
uniques after: [0 1]
dtype after: int64
nan count after: 0
----------working on: conduct_bims----------
uniques before: {'no', 'yes'}
dtype before: object
nan count before: 0
uniques after: [1 0]
dtype after: int64
nan count after: 0
----------working on: conduct_sams----------
uniques before: {'no', nan, 'yes'}
dtype before: object
nan count before: 3896
uniques after: [1 0]
dtype after: int64
nan count after: 0
----------working on: sams_season----------
uniques before: {'no', nan, 'yes'}
dtype before: object
nan count before: 39019
uniques after: [0 1]
dtype after: int64
nan count after: 0
----------working on: sams_room----------
uniques before: {'no', nan, 'yes'}
dtype befor

#### Column: all ranked integer features
**NaNs**: Leave them alone. Different imputation strategies can be applied later.

In [62]:
# safe buffer
last_df = df

In [63]:
# each case may need separate handling
df = last_df.copy() # always work on a fresh copy of last step's output

ranked_int_features = [
    "admityear",
    "hearing_adm",
    "vision_adm",
    "health_lit_adm",
    "expression_adm",
    "understand_verbal_adm",
    "bims_3words",
    "bims_year",
    "bims_month",
    "bims_day",
    "bims_recall_sock",
    "bims_recall_blue",
    "bims_recall_bed",
    "bims_total",
    "low_interest_adm",
    "low_interest_freq_adm",
    "depressed_adm",
    "depressed_freq_adm",
    "sleep_trouble_adm",
    "sleep_trouble_freq_adm",
    "tired_adm",
    "tired_freq_adm",
    "appetite_adm",
    "appetite_freq_adm",
    "feel_bad_adm",
    "feel_bad_freq_adm",
    "concentrate_adm",
    "concentrate_freq_adm",
    "slowfast_adm",
    "slowfast_freq_adm",
    "selfharm_adm",
    "selfharm_freq_adm",
    "mood_total_adm",
    "socisolation_adm",
    "selfcare_prior",
    "mobility_prior",
    "stairs_prior",
    "func_cog_prior",
    "eating_adm",
    "oral_adm",
    "toileting_adm",
    "bathe_adm",
    "dress_upper_adm",
    "dress_lower_adm",
    "footwear_adm",
    "roll_lr_adm",
    "sit_lying_adm",
    "lying_sit_adm",
    "sit_stand_adm",
    "bed_chair_adm",
    "toilet_trans_adm",
    "car_trans_adm",
    "walk10ft_adm",
    "walk50ft_adm",
    "walk150ft_adm",
    "walk10ft_uneven_adm",
    "walk1step_adm",
    "walk4step_adm",
    "walk12step_adm",
    "pickup_adm",
    "wheel50ft_adm",
    "wheel150ft_adm",
    "bladder_incontinence",
    "bowel_incontinence",
    "pain_sleep_adm",
    "pain_therapy_adm",
    "pain_activities_adm",
    "stage1_pu_adm",
    "stage2_pu_adm",
    "stage3_pu_adm",
    "stage4_pu_adm",
    "unstageable_dressing_pu_adm",
    "unstageable_slough_pu_adm",
    "unstageable_deep_pu_adm",
    "med_follow_up",
]

for name in ranked_int_features:
    ranked_int_feature_set.add(name)
    print(f"----------working on: {name}----------")
    print("uniques before:", df[name].unique())
    print("dtype before:", df[name].dtype)
    print("nan count before:", pd.isna(df[name]).sum())
    if name == "admityear": # simply replace float as int
        df[name] = df[name].apply(lambda x: int(x))
    elif name == "health_lit_adm":
        inverse_lookup = {v:k for k, v in lookup_table[name].items()}
        possible_nan_keys = ["Decline", "Unable", "Refuse"]
        for k in possible_nan_keys:
            inverse_lookup[k] = np.nan
        df[name].replace(inverse_lookup, inplace=True)
        df[name] = df[name].apply(lambda x: int(x) if pd.notna(x) else pd.NA).astype('Int64')
    elif name == "bims_total":
        df[name].replace({'unable_to_complete': pd.NA,
                          np.nan: pd.NA}, inplace=True)
        df[name] = df[name].apply(lambda x: int(float(x)) if pd.notna(x) else pd.NA).astype('Int64')
    else: # default behavior
        inverse_lookup = {v:k for k, v in lookup_table[name].items()}
        possible_nan_keys = [
            "Decline",
            "Unable", "unable"
            "Refuse",
            "No_response",
            "Unknown",
            "Not_applicable",
            "Not_attempted",
            "Safety",
            "Not_rated",
            ]
        for k in possible_nan_keys:
            inverse_lookup[k] = np.nan
        df[name].replace(inverse_lookup, inplace=True)
        df[name] = df[name].apply(lambda x: int(float(x)) if pd.notna(x) else pd.NA).astype('Int64')
    print("uniques after:", df[name].unique())
    print("dtype after:", df[name].dtype)
    print("nan count after:", pd.isna(df[name]).sum())
        

----------working on: admityear----------
uniques before: [2023. 2022.]
dtype before: float64
nan count before: 0
uniques after: [2023 2022]
dtype after: int64
nan count after: 0
----------working on: hearing_adm----------
uniques before: ['Adequate' 'Mod_difficulty' 'Min_difficulty' 'Highly_impaired']
dtype before: object
nan count before: 0
uniques after: <IntegerArray>
[0, 2, 1, 3]
Length: 4, dtype: Int64
dtype after: Int64
nan count after: 0
----------working on: vision_adm----------
uniques before: ['Adequate' 'Impaired' 'Mod_impaired' 'Highly_impaired'
 'Severely_impaired']
dtype before: object
nan count before: 0
uniques after: <IntegerArray>
[0, 1, 2, 3, 4]
Length: 5, dtype: Int64
dtype after: Int64
nan count after: 0
----------working on: health_lit_adm----------
uniques before: ['Never' 'Rarely' 'Unable' 'Often' 'Sometimes' 'Always' 'Decline']
dtype before: object
nan count before: 0
uniques after: <IntegerArray>
[0, 1, <NA>, 3, 2, 4]
Length: 6, dtype: Int64
dtype after: Int6

#### Column: all numeric columns
**NaNs**: Leave them alone. Different imputation strategies can be applied later.

In [64]:
# safe buffer
last_df = df

In [65]:
# replace nan with "Unknown", replace spaces with underscores
df = last_df.copy() # always work on a fresh copy of last step's output

numeric_features = [
    "heightinches",
    "weightpounds",
    "ptindweek1",
    "ptconweek1",
    "ptgrpweek1",
    "ptcoweek1",
    "otindweek1",
    "otconweek1",
    "otgrpweek1",
    "otcoweek1",
    "slpindweek1",
    "slpconweek1",
    "slpgrpweek1",
    "slpcoweek1",
    "ptindweek2",
    "ptconweek2",
    "ptgrpweek2",
    "ptcoweek2",
    "otindweek2",
    "otconweek2",
    "otgrpweek2",
    "otcoweek2",
    "slpindweek2",
    "slpconweek2",
    "slpgrpweek2",
    "slpcoweek2",
    "age_at_admit",
    "los",
    "ric",
    "cmg",
    "selfcare_measure_adm",
    "mobility_measure_adm"
]
# TODO

for name in numeric_features:
    numeric_feature_set.add(name)

#### Column: all categorical columns

**NaNs**: Simply set as "Unknwon". Later if imputation is needed you can still do so.

In [66]:
# safe buffer
last_df = df

In [67]:
# replace nan with "Unknown", replace spaces with underscores
df = last_df.copy() # always work on a fresh copy of last step's output

categorical_features = [
    "sex",
    "marital_status",
    "admitclass",
    "admit_from",
    "prehospital_living",
    "payor_primary",
    "arthritis",
    "inattention_adm",
    "disorganized_adm",
    "altered_adm",
    "wc50_type",
    "wc150_type",
    "falls_hx",
    "prior_surgery",
    "drug_regimen_review",
    "tier",
]   

for name in categorical_features:
    categorical_feature_set.add(name)
    print(f"----------working on: {name}----------")
    print("uniques before:", df[name].unique())
    print("dtype before:", df[name].dtype)
    print("nan count before:", pd.isna(df[name]).sum())
    if name in {"admitclass", "tier"}: # categorical but labels are integers
        df[name] = df[name].apply(lambda x: str(int(float(x))) if pd.notna(x) else "Unknown").astype(str)
    
    else: # default behavior
        df[name] = df[name].apply(lambda x: str(x).replace(' ', '_') if pd.notna(x) else "Unknown").astype(str)
    print("uniques after:", df[name].unique())
    print("dtype after:", df[name].dtype)
    print("nan count after:", pd.isna(df[name]).sum())

----------working on: sex----------
uniques before: ['female' 'male']
dtype before: object
nan count before: 0
uniques after: ['female' 'male']
dtype after: object
nan count after: 0
----------working on: marital_status----------
uniques before: ['Married' 'Widowed' nan 'Divorced' 'Separated' 'Never married']
dtype before: object
nan count before: 2533
uniques after: ['Married' 'Widowed' 'Unknown' 'Divorced' 'Separated' 'Never_married']
dtype after: object
nan count after: 0
----------working on: admitclass----------
uniques before: [ 1.  4.  3. nan  5.  2.]
dtype before: float64
nan count before: 3861
uniques after: ['1' '4' '3' 'Unknown' '5' '2']
dtype after: object
nan count after: 0
----------working on: admit_from----------
uniques before: ['Short-term general hospital' 'Home' 'Skilled nursing facility'
 'Home health service' 'Long-term care hospital' 'Not listed'
 'Another IRF' 'Intermediate care' 'Critical access hospital'
 'Medicaid nursing facility' 'Inpatient psychiatric faci

#### Save

In [68]:
# show processed features
feature_names = [*df.head()]
processed_feature_dtypes = df.dtypes
print("total # of features:", len(feature_names))
print(tabulate([(index, name, processed_feature_dtypes[name]) for index, name in enumerate(feature_names)], headers=["Index", "Name", "Dtype"], tablefmt="grid"))

total # of features: 455
+---------+--------------------------------+---------+
|   Index | Name                           | Dtype   |
+=========+================================+=========+
|       0 | sex                            | object  |
+---------+--------------------------------+---------+
|       1 | marital_status                 | object  |
+---------+--------------------------------+---------+
|       2 | admityear                      | int64   |
+---------+--------------------------------+---------+
|       3 | admitclass                     | object  |
+---------+--------------------------------+---------+
|       4 | admit_from                     | object  |
+---------+--------------------------------+---------+
|       5 | prehospital_living             | object  |
+---------+--------------------------------+---------+
|       6 | prehospital_living_with        | object  |
+---------+--------------------------------+---------+
|       7 | payor_primary               

In [74]:
# save processed data with 6 extra columns

df.to_csv('./data/Cleaned files/Cogan_1_3.csv', index=True)

In [34]:
# save feature table & dtype dictionary
import pickle
import json

feature_table = {
    "categorical": list(categorical_feature_set),
    "numeric": list(numeric_feature_set),
    "ranked_int": list(ranked_int_feature_set),
    "binary": list(binary_feature_set),
}

with open("./data/Cleaned files/feature_table.pkl", 'wb') as f:
    pickle.dump(feature_table, f)

dtype_dict = {name: str(df[name].dtype) for name in df.columns}

with open('./data/Cleaned files/processed_feature_dtypes.json', 'w') as f:
    json.dump(dtype_dict, f)

## 2 Selection, Imputation & One-Hot Expansion

### 2.1 Feature Selection & Dropping Invalid Samples & Splitting Dataset

In [3]:
# total # of features and a list of all feature names

import pandas as pd
import pickle
import numpy as np
from tabulate import tabulate
import json

def convert_dtypes(df, dtype_dict):
    for name in df.columns:
        dtype = dtype_dict[name]
        df[name] = df[name].astype(dtype)

# always start from loading the CSV file from previous section
df = pd.read_csv('./data/Cleaned files/Cogan_1_3.csv', low_memory=False, index_col='id')
with open("./data/Cleaned files/lookup_table.pkl", 'rb') as f:
    lookup_table = pickle.load(f)
with open("./data/Cleaned files/feature_table.pkl", 'rb') as f:
    feature_table = pickle.load(f)

# convert dtypes
with open('./data/Cleaned files/processed_feature_dtypes.json', 'r') as f:
    dtype_dict = json.load(f)
convert_dtypes(df, dtype_dict)

feature_names = [*df.head()]
feature_dtypes = df.dtypes
print("total # of features:", len(feature_names))

print(tabulate([(index, name, feature_dtypes[name]) for index, name in enumerate(feature_names)], headers=["Index", "Name", "Dtype"], tablefmt="grid"))
categorical_feature_set = feature_table["categorical"]
numeric_feature_set = feature_table["numeric"]
ranked_int_feature_set = feature_table["ranked_int"]
binary_feature_set = feature_table["binary"]

total # of features: 455
+---------+--------------------------------+---------+
|   Index | Name                           | Dtype   |
+=========+================================+=========+
|       0 | sex                            | object  |
+---------+--------------------------------+---------+
|       1 | marital_status                 | object  |
+---------+--------------------------------+---------+
|       2 | admityear                      | int64   |
+---------+--------------------------------+---------+
|       3 | admitclass                     | object  |
+---------+--------------------------------+---------+
|       4 | admit_from                     | object  |
+---------+--------------------------------+---------+
|       5 | prehospital_living             | object  |
+---------+--------------------------------+---------+
|       6 | prehospital_living_with        | object  |
+---------+--------------------------------+---------+
|       7 | payor_primary               

In [4]:
# define predictors and outcomes
interested_predictors = [
    # >>> base info >>>
    "sex", # male / female
    "marital_status", # ['Married' 'Widowed' 'Unknown' 'Divorced' 'Separated' 'Never_married']
    "admitclass",
    "admit_from",
    "prehospital_living",
    "payor_primary",
    # "impgroupadmit", # what does this do?
    # "diagnosis", # need a pretrained LM for embedding
    "arthritis", 
    "heightinches",
    "weightpounds",
    # <<< base info <<<

    # >>> treatments >>>
    "ptindweek1",
    "ptconweek1",
    "ptgrpweek1",
    "ptcoweek1",
    "otindweek1",
    "otconweek1",
    "otgrpweek1",
    "otcoweek1",
    "slpindweek1",
    "slpconweek1",
    "slpgrpweek1",
    "slpcoweek1",
    "ptindweek2",
    "ptconweek2",
    "ptgrpweek2",
    "ptcoweek2",
    "otindweek2",
    "otconweek2",
    "otgrpweek2",
    "otcoweek2",
    "slpindweek2",
    "slpconweek2",
    "slpgrpweek2",
    "slpcoweek2",
    # <<< treatments <<<

    # >>> section A admission >>>
    "hispanic", # ethnicity, Yes or No
    "race", # integrated race categorical field
    "language", # integrated language categorical field: English or Non-English
    "transport_lack_med", # NACHC
    "transport_lack_nonmed", # NACHC
    # <<< section A admission <<<
    
    # >>> section B C admission >>>
    "hearing_adm",
    "vision_adm",
    "health_lit_adm",
    "expression_adm",
    "understand_verbal_adm",
    "conduct_bims", # bims: Brief Interview for Mental Status
    # "bims_3words",
    # "bims_year",
    # "bims_month",
    # "bims_day",
    # "bims_recall_sock",
    # "bims_recall_blue",
    # "bims_recall_bed",
    "bims_total",
    "conduct_sams", # only if "conduct_bims" is false, conduct Staff Assessment for Mental Status
    "sams_season",
    "sams_room",
    "sams_names",
    "sams_hosp",
    # "sams_none_above", # don't include this since it's inferable from others
    "acute_mental_change",
    "inattention_adm",
    "disorganized_adm",
    "altered_adm",
    # <<< section B C admission <<<
    
    # >>> section D admission >>>
    # "low_interest_adm",
    # "low_interest_freq_adm",
    # "depressed_adm",
    # "depressed_freq_adm",
    # "sleep_trouble_adm",
    # "sleep_trouble_freq_adm",
    # "tired_adm",
    # "tired_freq_adm",
    # "appetite_adm",
    # "appetite_freq_adm",
    # "feel_bad_adm",
    # "feel_bad_freq_adm",
    # "concentrate_adm",
    # "concentrate_freq_adm",
    # "slowfast_adm",
    # "slowfast_freq_adm",
    # "selfharm_adm",
    # "selfharm_freq_adm",
    "mood_total_adm", # total alone is enough to cover all above
    "socisolation_adm",
    # <<< section D admission <<<

    # >>> section GG admission >>>
    "selfcare_prior",
    "mobility_prior",
    "stairs_prior",
    "func_cog_prior",
    "wc_manual_prior",
    "wc_motor_prior",
    "mechlift_prior",
    "walker_prior",
    "orth_pros_prior",
    # "no_device_prior", # can be inferred
    # "eating_adm",
    # # "eating_goal",
    # "oral_adm",
    # # "oral_goal",
    # "toileting_adm",
    # # "toileting_goal",
    # "bathe_adm",
    # # "bathe_goal",
    # "dress_upper_adm",
    # # "dress_upper_goal",
    # "dress_lower_adm",
    # # "dress_lower_goal",
    # "footwear_adm",
    # # "footwear_goal",
    # "roll_lr_adm",
    # # "roll_lr_goal",
    # "sit_lying_adm",
    # # "sit_lying_goal",
    # "lying_sit_adm",
    # # "lying_sit_goal",
    # "sit_stand_adm",
    # # "sit_stand_goal",
    # "bed_chair_adm",
    # # "bed_chair_goal",
    # "toilet_trans_adm",
    # # "toilet_trans_goal",
    # "car_trans_adm",
    # # "car_trans_goal",
    # "walk10ft_adm",
    # # "walk10ft_goal",
    # "walk50ft_adm",
    # # "walk50ft_goal",
    # "walk150ft_adm",
    # # "walk150ft_goal",
    # "walk10ft_uneven_adm",
    # # "walk10ft_uneven_goal",
    # "walk1step_adm",
    # # "walk1step_goal",
    # "walk4step_adm",
    # # "walk4step_goal",
    # "walk12step_adm",
    # # "walk12step_goal",
    # "pickup_adm",
    # # "pickup_goal",
    "selfcare_measure_adm", # added in section 1.2
    "mobility_measure_adm", # added in section 1.2
    "wc_user", # inferable, but still preserved
    "wheel50ft_adm",
    # "wheel50ft_goal",
    "wc50_type",
    "wheel150ft_adm",
    # "wheel150ft_goal",
    "wc150_type",
    # <<< section GG admission <<<

    # >>> section H I J K M N O admission >>>
    "bladder_incontinence",
    "bowel_incontinence",
    "pvd_comorbid",
    "diabetes_comorbid",
    # "no_comorbidities", # inferable
    "pain_sleep_adm",
    "pain_therapy_adm",
    "pain_activities_adm",
    "falls_hx",
    "prior_surgery",
    "nutrition_parenteral_adm",
    "nutrition_tube_adm",
    "nutrition_mech_diet_adm",
    "nutrition_ther_diet_adm",
    # "nutrition_none_adm", # inferable
    # "pressure_ulcer_adm", #inferable
    "stage1_pu_adm",
    "stage2_pu_adm",
    "stage3_pu_adm",
    "stage4_pu_adm",
    "unstageable_dressing_pu_adm",
    "unstageable_slough_pu_adm",
    "unstageable_deep_pu_adm",
    "antipsychotic_taking_adm",
    "antipsychotic_ind_adm",
    "anticoagulant_taking_adm",
    "anticoagulant_ind_adm",
    "antibiotic_taking_adm",
    "antibiotic_ind_adm",
    "opioid_taking_adm",
    "opioid_ind_adm",
    "antiplatelet_taking_adm",
    "antiplatelet_ind_adm",
    "hypoglycemic_taking_adm",
    "hypoglycemic_ind_adm",
    # "med_highrisk_none", # inferable
    "drug_regimen_review",
    "med_follow_up",
    "chemo_adm",
    "chemo_iv_adm",
    "chemo_oral_adm",
    "chemo_other_adm",
    "radiation_adm",
    "oxygen_adm",
    "oxygen_cont_adm",
    "oxygen_int_adm",
    "oxygen_high_adm",
    "suctioning_adm",
    "suctioning_sched_adm",
    "suctioning_asneeded_adm",
    "trach_adm",
    "vent_invasive_adm",
    "vent_noninvasive_adm",
    "vent_bipap_adm",
    "vent_cpap_adm",
    "meds_iv_adm",
    "meds_iv_vasoactive_adm",
    "meds_iv_antibiotic_adm",
    "meds_iv_anticoagulant_adm",
    "meds_iv_other_adm",
    "transfusions_adm",
    "dialysis_adm",
    "hemodialysis_adm",
    "peritoneal_dialysis_adm",
    "iv_access_adm",
    "iv_access_periph_adm",
    "iv_access_mid_adm",
    "iv_access_cent_adm",
    # "tx_none_adm", # inferable
    # <<< section H I J K M N O admission <<<

    # >>> the rest >>>
    "age_at_admit",
    "los",
    "ric",
    "cmg",
    "tier",
    # "shortstayexpired",
    # "shortstaycmg",
    # "transferpatient",
    # "incompletestay",
    # <<< the rest <<<

]

interested_outcomes = [
    "selfcare_measure_delta", # added in section 1.2
    "mobility_measure_delta" # added in section 1.2
]

In [5]:
# cross-verify feature table

all_features_in_table = set()
for k in feature_table.keys():
    for name in feature_table[k]:
        all_features_in_table.add(name)
        if name not in interested_predictors+interested_outcomes:
            print(f"{name} is processed but not selected!")

for name in interested_predictors+interested_outcomes:
    if name not in all_features_in_table:
        print(f"{name} is interesting but not processed!")

depressed_adm is processed but not selected!
sit_stand_adm is processed but not selected!
depressed_freq_adm is processed but not selected!
sit_lying_adm is processed but not selected!
walk150ft_adm is processed but not selected!
dress_upper_adm is processed but not selected!
bed_chair_adm is processed but not selected!
toileting_adm is processed but not selected!
feel_bad_freq_adm is processed but not selected!
walk10ft_adm is processed but not selected!
admityear is processed but not selected!
appetite_adm is processed but not selected!
car_trans_adm is processed but not selected!
slowfast_freq_adm is processed but not selected!
bims_recall_bed is processed but not selected!
appetite_freq_adm is processed but not selected!
selfharm_freq_adm is processed but not selected!
footwear_adm is processed but not selected!
walk4step_adm is processed but not selected!
pickup_adm is processed but not selected!
sleep_trouble_freq_adm is processed but not selected!
walk1step_adm is processed but 

In [6]:
# select subset of features
df = df[interested_outcomes+interested_predictors]

feature_names = [*df.head()]
feature_dtypes = df.dtypes
print("total # of features:", len(feature_names))
print(tabulate([(index, name, feature_dtypes[name]) for index, name in enumerate(feature_names)], headers=["Index", "Name", "Dtype"], tablefmt="grid"))

total # of features: 143
+---------+-----------------------------+---------+
|   Index | Name                        | Dtype   |
+=========+=============================+=========+
|       0 | selfcare_measure_delta      | float64 |
+---------+-----------------------------+---------+
|       1 | mobility_measure_delta      | float64 |
+---------+-----------------------------+---------+
|       2 | sex                         | object  |
+---------+-----------------------------+---------+
|       3 | marital_status              | object  |
+---------+-----------------------------+---------+
|       4 | admitclass                  | object  |
+---------+-----------------------------+---------+
|       5 | admit_from                  | object  |
+---------+-----------------------------+---------+
|       6 | prehospital_living          | object  |
+---------+-----------------------------+---------+
|       7 | payor_primary               | object  |
+---------+----------------------------

In [7]:
# samples with both "selfcare_measure_delta" and "mobility_measure_delta" being np.nan are useless, drop them

df.dropna(subset=interested_outcomes, how='all', inplace=True)
print(f"# of remaining rows: {df.shape[0]}")

# of remaining rows: 33519


In [8]:
# save selected csv
df.to_csv('./data/Cleaned files/Cogan_2_1.csv', index=True)

In [9]:
# create dataset splits

TRAIN_RATIO = 4
EVAL_RATIO = 1
TEST_RATIO = 1

indices = df.index.tolist()
np.random.seed(42)  # for reproducibility
np.random.shuffle(indices)

sum_ratios = TRAIN_RATIO + EVAL_RATIO + TEST_RATIO
total_size = len(indices)
train_size = int(TRAIN_RATIO / sum_ratios * total_size)  # 4/6th for train
eval_size = int(EVAL_RATIO / sum_ratios * total_size)   # 1/6th for eval
test_size = total_size - train_size - eval_size  # Remaining for test

train_indices = indices[:train_size]
eval_indices = indices[train_size:train_size + eval_size]
test_indices = indices[train_size + eval_size:]

split_dict = {
    "train": train_indices,
    "eval": eval_indices,
    "test": test_indices
}

with open('./data/Cleaned files/data_splits.json', 'w') as json_file:
    json.dump(split_dict, json_file)

### 2.2 Imputation


In [38]:
# total # of features and a list of all feature names

import pandas as pd
import pickle
import numpy as np
from tabulate import tabulate
import json

def convert_dtypes(df, dtype_dict):
    for name in df.columns:
        dtype = dtype_dict[name]
        df[name] = df[name].astype(dtype)

# always start from loading the CSV file from previous section
df = pd.read_csv('./data/Cleaned files/Cogan_2_1.csv', low_memory=False, index_col='id')
with open("./data/Cleaned files/lookup_table.pkl", 'rb') as f:
    lookup_table = pickle.load(f)
with open("./data/Cleaned files/feature_table.pkl", 'rb') as f:
    feature_table = pickle.load(f)

# convert dtypes
with open('./data/Cleaned files/processed_feature_dtypes.json', 'r') as f:
    dtype_dict = json.load(f)
convert_dtypes(df, dtype_dict)

# load splits
with open('./data/Cleaned files/data_splits.json', 'r') as f:
    split_dict = json.load(f)

print("# of train samples:", len(split_dict["train"]))
print("# of eval samples:", len(split_dict["eval"]))
print("# of test samples:", len(split_dict["test"]))

print("# of valid selfcare train samples:", df.loc[split_dict["train"]]['selfcare_measure_delta'].notna().sum())
print("# of valid selfcare eval samples:", df.loc[split_dict["eval"]]['selfcare_measure_delta'].notna().sum())
print("# of valid selfcare test samples:", df.loc[split_dict["test"]]['selfcare_measure_delta'].notna().sum())

print("# of valid mobility train samples:", df.loc[split_dict["train"]]['mobility_measure_delta'].notna().sum())
print("# of valid mobility eval samples:", df.loc[split_dict["eval"]]['mobility_measure_delta'].notna().sum())
print("# of valid mobility test samples:", df.loc[split_dict["test"]]['mobility_measure_delta'].notna().sum())


feature_names = [*df.head()]
feature_dtypes = df.dtypes
print("total # of features:", len(feature_names))

print(tabulate([(index, name, feature_dtypes[name]) for index, name in enumerate(feature_names)], headers=["Index", "Name", "Dtype"], tablefmt="grid"))
categorical_feature_set = feature_table["categorical"]
numeric_feature_set = feature_table["numeric"]
ranked_int_feature_set = feature_table["ranked_int"]
binary_feature_set = feature_table["binary"]
outcomes = ["selfcare_measure_delta", "mobility_measure_delta"]

# of train samples: 22346
# of eval samples: 5586
# of test samples: 5587
# of valid selfcare train samples: 22165
# of valid selfcare eval samples: 5538
# of valid selfcare test samples: 5552
# of valid mobility train samples: 2482
# of valid mobility eval samples: 621
# of valid mobility test samples: 540
total # of features: 143
+---------+-----------------------------+---------+
|   Index | Name                        | Dtype   |
+=========+=============================+=========+
|       0 | selfcare_measure_delta      | float64 |
+---------+-----------------------------+---------+
|       1 | mobility_measure_delta      | float64 |
+---------+-----------------------------+---------+
|       2 | sex                         | object  |
+---------+-----------------------------+---------+
|       3 | marital_status              | object  |
+---------+-----------------------------+---------+
|       4 | admitclass                  | object  |
+---------+-----------------------------+-

#### 2.2.1 Default Strategy

In [39]:
# safe buffer
last_df = df

In [40]:
# ranked int & categorical features: mode imputation

df = last_df.copy() # always work on a fresh copy of last step's output

train_indices = split_dict['train']

train_df = df.loc[train_indices]

for name in interested_predictors:
    if name in categorical_feature_set or name in ranked_int_feature_set:
        mode_value = train_df[name].mode()[0]  # Get the most frequent value in the training set
        df[name].fillna(mode_value, inplace=True)
    elif name in numeric_feature_set:
        mean_value = train_df[name].mean()
        df[name].fillna(mean_value, inplace=True)
        df[name] = df[name].round(2)

print("# of nans in all predictors left:", df[interested_predictors].isna().sum().sum())


# of nans in all predictors left: 0


In [41]:
# save df

df.to_csv('./data/Cleaned files/Cogan_2_2.csv', index=True)

### 2.3 One-hot expansion

In [48]:
# total # of features and a list of all feature names

import pandas as pd
import pickle
import numpy as np
from tabulate import tabulate
import json

def convert_dtypes(df, dtype_dict):
    for name in df.columns:
        dtype = dtype_dict[name]
        df[name] = df[name].astype(dtype)

# always start from loading the CSV file from previous section
df = pd.read_csv('./data/Cleaned files/Cogan_2_2.csv', low_memory=False, index_col='id')
with open("./data/Cleaned files/lookup_table.pkl", 'rb') as f:
    lookup_table = pickle.load(f)
with open("./data/Cleaned files/feature_table.pkl", 'rb') as f:
    feature_table = pickle.load(f)

# convert dtypes
with open('./data/Cleaned files/processed_feature_dtypes.json', 'r') as f:
    dtype_dict = json.load(f)
convert_dtypes(df, dtype_dict)

# load splits
with open('./data/Cleaned files/data_splits.json', 'r') as f:
    split_dict = json.load(f)

print("# of train samples:", len(split_dict["train"]))
print("# of eval samples:", len(split_dict["eval"]))
print("# of test samples:", len(split_dict["test"]))

print("# of valid selfcare train samples:", df.loc[split_dict["train"]]['selfcare_measure_delta'].notna().sum())
print("# of valid selfcare eval samples:", df.loc[split_dict["eval"]]['selfcare_measure_delta'].notna().sum())
print("# of valid selfcare test samples:", df.loc[split_dict["test"]]['selfcare_measure_delta'].notna().sum())

print("# of valid mobility train samples:", df.loc[split_dict["train"]]['mobility_measure_delta'].notna().sum())
print("# of valid mobility eval samples:", df.loc[split_dict["eval"]]['mobility_measure_delta'].notna().sum())
print("# of valid mobility test samples:", df.loc[split_dict["test"]]['mobility_measure_delta'].notna().sum())


feature_names = [*df.head()]
feature_dtypes = df.dtypes
print("total # of features:", len(feature_names))

nan_counts = df.isna().sum(axis=0).to_dict()


def tabulate_df(df):
    feature_names = [*df.head()]
    feature_dtypes = df.dtypes
    nan_counts = df.isna().sum(axis=0).to_dict()
    return tabulate([(index, name, feature_dtypes[name], nan_counts[name]) for index, name in enumerate(feature_names)], headers=["Index", "Name", "Dtype", "# of NaNs"], tablefmt="grid")

print(tabulate_df(df))
categorical_feature_set = feature_table["categorical"]
numeric_feature_set = feature_table["numeric"]
ranked_int_feature_set = feature_table["ranked_int"]
binary_feature_set = feature_table["binary"]
outcomes = ["selfcare_measure_delta", "mobility_measure_delta"]

# of train samples: 22346
# of eval samples: 5586
# of test samples: 5587
# of valid selfcare train samples: 22165
# of valid selfcare eval samples: 5538
# of valid selfcare test samples: 5552
# of valid mobility train samples: 2482
# of valid mobility eval samples: 621
# of valid mobility test samples: 540
total # of features: 143
+---------+-----------------------------+---------+-------------+
|   Index | Name                        | Dtype   |   # of NaNs |
+=========+=============================+=========+=============+
|       0 | selfcare_measure_delta      | float64 |         264 |
+---------+-----------------------------+---------+-------------+
|       1 | mobility_measure_delta      | float64 |       29876 |
+---------+-----------------------------+---------+-------------+
|       2 | sex                         | object  |           0 |
+---------+-----------------------------+---------+-------------+
|       3 | marital_status              | object  |           0 |
+-----

In [49]:
# safe buffer
last_df = df

In [50]:
# expand all categorical features to n_class-1 one-hot representations
df = last_df.copy()

columns_to_one_hot = []
for name in df.columns:
    if name in categorical_feature_set:
        columns_to_one_hot.append(name)
df = pd.get_dummies(df, columns=columns_to_one_hot, drop_first=True)
df = df.astype({col: 'int' for col in df.select_dtypes('bool').columns}) # 0/1 instead of False/True

print(tabulate_df(df))

+---------+---------------------------------------------------+---------+-------------+
|   Index | Name                                              | Dtype   |   # of NaNs |
+=========+===================================================+=========+=============+
|       0 | selfcare_measure_delta                            | float64 |         264 |
+---------+---------------------------------------------------+---------+-------------+
|       1 | mobility_measure_delta                            | float64 |       29876 |
+---------+---------------------------------------------------+---------+-------------+
|       2 | heightinches                                      | float64 |           0 |
+---------+---------------------------------------------------+---------+-------------+
|       3 | weightpounds                                      | float64 |           0 |
+---------+---------------------------------------------------+---------+-------------+
|       4 | ptindweek1          

In [51]:
# save

df.to_csv('./data/Cleaned files/Cogan_2_3.csv', index=True)

## 3 Analysis

In [2]:
# load processed data, do a tabulate

# total # of features and a list of all feature names

import pandas as pd
import pickle
import numpy as np
from tabulate import tabulate
import json

# always start from loading the CSV file from previous section
df = pd.read_csv('./data/Cleaned files/Cogan_2_3.csv', low_memory=False, index_col='id')

# load splits
with open('./data/Cleaned files/data_splits.json', 'r') as f:
    split_dict = json.load(f)

print("# of train samples:", len(split_dict["train"]))
print("# of eval samples:", len(split_dict["eval"]))
print("# of test samples:", len(split_dict["test"]))

print("# of valid selfcare train samples:", df.loc[split_dict["train"]]['selfcare_measure_delta'].notna().sum())
print("# of valid selfcare eval samples:", df.loc[split_dict["eval"]]['selfcare_measure_delta'].notna().sum())
print("# of valid selfcare test samples:", df.loc[split_dict["test"]]['selfcare_measure_delta'].notna().sum())

print("# of valid mobility train samples:", df.loc[split_dict["train"]]['mobility_measure_delta'].notna().sum())
print("# of valid mobility eval samples:", df.loc[split_dict["eval"]]['mobility_measure_delta'].notna().sum())
print("# of valid mobility test samples:", df.loc[split_dict["test"]]['mobility_measure_delta'].notna().sum())


feature_names = [*df.head()]
feature_dtypes = df.dtypes
print("total # of features:", len(feature_names))

nan_counts = df.isna().sum(axis=0).to_dict()


def tabulate_df(df):
    feature_names = [*df.head()]
    feature_dtypes = df.dtypes
    nan_counts = df.isna().sum(axis=0).to_dict()
    return tabulate([(index, name, feature_dtypes[name], nan_counts[name]) for index, name in enumerate(feature_names)], headers=["Index", "Name", "Dtype", "# of NaNs"], tablefmt="grid")

print(tabulate_df(df))

predictors = [name for name in df.columns if name not in ['id', 'selfcare_measure_delta', 'mobility_measure_delta']]


# of train samples: 22346
# of eval samples: 5586
# of test samples: 5587
# of valid selfcare train samples: 22165
# of valid selfcare eval samples: 5538
# of valid selfcare test samples: 5552
# of valid mobility train samples: 2482
# of valid mobility eval samples: 621
# of valid mobility test samples: 540
total # of features: 187
+---------+---------------------------------------------------+---------+-------------+
|   Index | Name                                              | Dtype   |   # of NaNs |
+=========+===================================================+=========+=============+
|       0 | selfcare_measure_delta                            | float64 |         264 |
+---------+---------------------------------------------------+---------+-------------+
|       1 | mobility_measure_delta                            | float64 |       29876 |
+---------+---------------------------------------------------+---------+-------------+
|       2 | heightinches                          

### 3.1 Selfcare

In [3]:
# drop invalid samples & split dataset

selfcare_df = df.dropna(subset=["selfcare_measure_delta"]).copy()

selfcare_split_dict = {} # since some rows are dropped, we need to filter the split dict
for k in split_dict.keys():
    selfcare_split_dict[k] = [idx for idx in split_dict[k] if idx in selfcare_df.index]

selfcare_train_df = selfcare_df.loc[selfcare_split_dict["train"]]
selfcare_eval_df = selfcare_df.loc[selfcare_split_dict["eval"]]
selfcare_test_df = selfcare_df.loc[selfcare_split_dict["test"]]

X_train = selfcare_train_df[predictors]
y_train = selfcare_train_df['selfcare_measure_delta']

X_eval = selfcare_eval_df[predictors]
y_eval = selfcare_eval_df['selfcare_measure_delta']

X_test = selfcare_test_df[predictors]
y_test = selfcare_test_df['selfcare_measure_delta']

#### 3.1.1 Random Forest

In [14]:
# RF

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# fit the model on the train set
rf_regressor.fit(X_train, y_train)

# make predictions on all sets
y_train_pred = rf_regressor.predict(X_train)
y_eval_pred = rf_regressor.predict(X_eval)
y_test_pred = rf_regressor.predict(X_test)

# train set performance
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# eval set performance
eval_mse = mean_squared_error(y_eval, y_eval_pred)
eval_r2 = r2_score(y_eval, y_eval_pred)

# test set performance
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("----------train set results----------")
print(f"MSE: {train_mse}")
print(f"R²: {train_r2}")

print("----------eval set results----------")
print(f"MSE: {eval_mse}")
print(f"R²: {eval_r2}")


print("----------test set results----------")
print(f"MSE: {test_mse}")
print(f"R²: {test_r2}")


----------train set results----------
MSE: 36.690154874635695
R²: 0.9044738752962913
----------eval set results----------
MSE: 270.6174670886728
R²: 0.32237085238452734
----------test set results----------
MSE: 253.71850463961096
R²: 0.3362843284484136


#### 3.1.2 XGBoost

In [5]:
# xgboost

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

xgb_regressor = XGBRegressor(n_estimators=100, max_depth=6, random_state=42)

xgb_regressor.fit(X_train, y_train)

y_train_pred = xgb_regressor.predict(X_train)
y_eval_pred = xgb_regressor.predict(X_eval)
y_test_pred = xgb_regressor.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

eval_mse = mean_squared_error(y_eval, y_eval_pred)
eval_r2 = r2_score(y_eval, y_eval_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("----------train set results----------")
print(f"MSE: {train_mse}")
print(f"R²: {train_r2}")

print("----------eval set results----------")
print(f"MSE: {eval_mse}")
print(f"R²: {eval_r2}")


print("----------test set results----------")
print(f"MSE: {test_mse}")
print(f"R²: {test_r2}")


----------train set results----------
MSE: 143.34259904125895
R²: 0.6267946254749903
----------eval set results----------
MSE: 275.34508955776096
R²: 0.3105328331375057
----------test set results----------
MSE: 251.22332818494507
R²: 0.3428115926643234


In [16]:
from tabulate import tabulate

booster = xgb_regressor.get_booster()
importance_dict = booster.get_score(importance_type='gain')

importance_list = [(feature, score) for feature, score in importance_dict.items()]

importance_list = sorted(importance_list, key=lambda x: x[1], reverse=True)

print(tabulate(importance_list, headers=["Feature", "Gain"], tablefmt="pretty"))

+---------------------------------------------------+--------------------+
|                      Feature                      |        Gain        |
+---------------------------------------------------+--------------------+
|                        cmg                        |   30233.0859375    |
|               understand_verbal_adm               |  6846.49169921875  |
|                    sams_season                    |  5672.2978515625   |
|                 wc150_type_manual                 |  5608.56787109375  |
|                     sams_hosp                     |  5066.82470703125  |
|                  selfcare_prior                   |  4461.4912109375   |
|            inattention_adm_Not_present            |  4296.13525390625  |
|               selfcare_measure_adm                | 3720.903076171875  |
|               bladder_incontinence                | 3561.692138671875  |
|                    bims_total                     |  3388.77490234375  |
|                  mobili

### 3.2 Mobility

In [9]:
# drop invalid samples & split dataset

mobility_df = df.dropna(subset=["mobility_measure_delta"]).copy()

mobility_split_dict = {} # since some rows are dropped, we need to filter the split dict
for k in split_dict.keys():
    mobility_split_dict[k] = [idx for idx in split_dict[k] if idx in mobility_df.index]

mobility_train_df = mobility_df.loc[mobility_split_dict["train"]]
mobility_eval_df = mobility_df.loc[mobility_split_dict["eval"]]
mobility_test_df = mobility_df.loc[mobility_split_dict["test"]]

X_train = mobility_train_df[predictors]
y_train = mobility_train_df['mobility_measure_delta']

X_eval = mobility_eval_df[predictors]
y_eval = mobility_eval_df['mobility_measure_delta']

X_test = mobility_test_df[predictors]
y_test = mobility_test_df['mobility_measure_delta']

#### 3.2.1 Random Forest

In [12]:
# RF

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# fit the model on the train set
rf_regressor.fit(X_train, y_train)

# make predictions on all sets
y_train_pred = rf_regressor.predict(X_train)
y_eval_pred = rf_regressor.predict(X_eval)
y_test_pred = rf_regressor.predict(X_test)

# train set performance
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# eval set performance
eval_mse = mean_squared_error(y_eval, y_eval_pred)
eval_r2 = r2_score(y_eval, y_eval_pred)

# test set performance
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("----------train set results----------")
print(f"MSE: {train_mse}")
print(f"R²: {train_r2}")

print("----------eval set results----------")
print(f"MSE: {eval_mse}")
print(f"R²: {eval_r2}")


print("----------test set results----------")
print(f"MSE: {test_mse}")
print(f"R²: {test_r2}")


----------train set results----------
MSE: 53.15796503378327
R²: 0.8842451996729045
----------eval set results----------
MSE: 397.6663327581159
R²: 0.1644848063638168
----------test set results----------
MSE: 394.8017951380926
R²: 0.1845204627916619


#### 3.2.2 XGBoost

In [10]:
# xgboost

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

xgb_regressor = XGBRegressor(n_estimators=100, max_depth=6, random_state=42)

xgb_regressor.fit(X_train, y_train)

y_train_pred = xgb_regressor.predict(X_train)
y_eval_pred = xgb_regressor.predict(X_eval)
y_test_pred = xgb_regressor.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

eval_mse = mean_squared_error(y_eval, y_eval_pred)
eval_r2 = r2_score(y_eval, y_eval_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("----------train set results----------")
print(f"MSE: {train_mse}")
print(f"R²: {train_r2}")

print("----------eval set results----------")
print(f"MSE: {eval_mse}")
print(f"R²: {eval_r2}")


print("----------test set results----------")
print(f"MSE: {test_mse}")
print(f"R²: {test_r2}")


----------train set results----------
MSE: 25.431102541450507
R²: 0.9446221804218304
----------eval set results----------
MSE: 441.0375325948382
R²: 0.07335992742704689
----------test set results----------
MSE: 427.68975830359517
R²: 0.11658900626789392


In [11]:
from tabulate import tabulate

booster = xgb_regressor.get_booster()
importance_dict = booster.get_score(importance_type='gain')

importance_list = [(feature, score) for feature, score in importance_dict.items()]

importance_list = sorted(importance_list, key=lambda x: x[1], reverse=True)

print(tabulate(importance_list, headers=["Feature", "Gain"], tablefmt="pretty"))

+----------------------------------------+--------------------+
|                Feature                 |        Gain        |
+----------------------------------------+--------------------+
|         prior_surgery_Unknown          | 2929.736083984375  |
|             selfcare_prior             |  2873.9560546875   |
|         meds_iv_antibiotic_adm         |      2826.875      |
|      inattention_adm_Not_present       | 2669.939208984375  |
|              stairs_prior              |     2483.9375      |
|             opioid_ind_adm             | 2074.991943359375  |
|          hypoglycemic_ind_adm          | 1771.2899169921875 |
|             func_cog_prior             |  1586.89404296875  |
|                  cmg                   |  1581.15576171875  |
|          language_Non-English          |   1473.361328125   |
|         understand_verbal_adm          |  1307.69091796875  |
|              walker_prior              | 1275.4822998046875 |
|          iv_access_periph_adm         